<img src="Images/IMG-dirty-data.png" style="display: block;
  margin-left: auto;
  margin-right: auto;
  width: 50%;">

Data processing of any kind (statistical analysis, machine learning, visualizations, ...) requires neatly and consistently structured data. In published dataset, much work has usually been done to make sure that all data points adhere to a common format and structure so that whatever processing we plan on doing can start right away. 

Often, however, and especially when we collect our own data, the datapoints can be a bit of a mess: Maybe we tapped different sources, which delivered data in different formats. Maybe some of our data points are incomplete. Maybe we have to convert some of our data to be able to process it. In a word: our data is dirty!

In this session, we will look at some selected techniques to deal with Dirty Data in a systematic and effective way. 

<img style="float: right; margin:5px 0px 0px 10px" src="Images/IMG-lego-logo.svg" alt="Lego-Logo" height="128" width="128" align="right">

The example data we will be handling are various facts and figures about Lego sets. It was collected by scraping the public product descriptions of www.lego.com. If you are interested in how you can collect data in this way, look out for our workshop on "Scraping and using web APIs with Python"!

In the course of this notebook, we will consolidate the data from two different sets of files, analyze its structure and format, clean up incomplete entries, and store the clean dataset for later use. This process is often called **Data Wrangling** or **Data Munging**.

# What you will learn in this session
- Importing data from inconsistent sources
- Cleaning up inconsistent or inconveniently formatted entries
- Dealing with missing values

We will use the Python data analysis and manipulation package [pandas](https://pandas.pydata.org/), but the presented strategies can be applied in any programming language and framework.



## Before we get started
There a few pandas-specific things you will see repeatedly in the examples that do not directly relate to the data cleaning techniques but are due to some of pandas quirks. We will discuss them here briefly:
### The `DataFrame` object
Everything in pandas revolves around its core data structure: The `DataFrame`. You can think of a data frame as a big table that has a column for each variable and a row for each observation. Our first objective will be to get our data, which lives in various files, into a single `DataFrame` object! 

### The `Series` object
Besides the `DataFrame` object, there is another structure to store strictly one-dimensional data (so either a single row or a single column): the `Series` object. Pandas will usually be smart enough to figure out which one to use on its own, but sometimes we have to be give it a hint (for example when loading a single row of data from a file).

### Functions versus methods
Almost all of the analysis and manipulation will be accomplished by calling code built into pandas. You may be familiar with the concept of calling functions from a package or library, where you pass everything to the function and it then works its magic:

```python
head(dataframe)  # This is *not* how it works in pandas!
```

In pandas, however, you mostly call *methods* instead of functions, which simply means that they are called on the `DataFrame` itself:

```python
dataframe.head()  # The method is provided by the DataFrame object!
```

This approach is part of the *object-oriented paradigm* and has a number of advantages. If you are interested in hearing more about object-oriented programming and how it may help you to write more readable and robust code, let us know! 

### The `inplace` keyword
Having heard about the difference between functions and methods, you may be wondering: What if our function or method has some kind of output? A function simply returns it:

```python
mean_value = mean(dataframe)  # This is *not* how it works in pandas
```

And the `DataFrame`'s methods usually do that, too:

```python
mean_value = dataframe.mean()  # An example method returning a value
```

In some cases, that means that a method returns a manipulated copy of the entire data frame:
```python
smaller_dataframe = dataframe.drop(columns='some_column_name')  # An example method returning a new manipulated copy of the dataframe
```

Sometimes we want a copy because we want to keep working both the manipulated and the original version. Often, however, we perform a series of manipulations and are only interested in the final version of the dataframe. So we do not want to create a bunch of copies along the way! 

For this use case, many methods provide the keyword `inplace`, which we can use to manipulate the dataframe in-place. In that case, no copy is made and the method returns nothing (the `None` object, to be precise), but the changes are applied to the original dataframe:

```python
dataframe.drop(columns='some_column_name', inplace=True)  # The column is removed from the original dataframe, no copy is made!
```

### Accessing columns
As mentioned above, `DataFrame` objects usually contain multiple columns. You can access |individual columns in multiple ways. Two of the most convenient ones are the following:

```python
dataframe['column_name']  # Note the quotes: The column name is provided as a string!
dataframe.column_name     # Note the lack of quotes: The column name is treated like a field name!
```
These two methods are functionally totally equivalent. They each have their advantages. The square bracket notation, for example, lets you select multiple columns:
```python
dataframe[['column_name_1', 'column_name_2']]  # Note the nested square brackets: multiple column names are passed as a list!
```
You can only select a single column using the dot notation, but it is a bit more convenient to type and arguably more readable. There is no need to exclusively use only one or the other and you typically see both ways of accessing columns in a pandas project.

# 1. Import Data

First of all, we need to download our dataset from [its repository](https://git.dartmouth.edu/lib-digital-strategies/RDS/lego-dataset). You could download and unzip it manually or we can use Python:

In [ ]:
"""
Download the dataset as a zipped archive
"""
import urllib.request
urllib.request.urlretrieve("https://git.dartmouth.edu/lib-digital-strategies/RDS/lego-dataset/-/archive/main/lego-dataset-main.zip", "dataset.zip")

"""
Unzip the files
"""
import zipfile
with zipfile.ZipFile("dataset.zip", 'r') as zf:
    zf.extractall()

"""
Clean up after ourselves
"""
import os
os.remove("dataset.zip")

Our data does not come in a single file. Instead, the data of every Lego set were saved in their own separate file. Even worse, the files came from different sources. We cannot be sure that they contain the same kinds of data and even if they do, they might use different formats. At the very least, the fact that the filenames apparently follow different naming conventions should make us suspicious. Let's investigate!

First, we need to load all the individual files and consolidate them in a dataframe. Since there are two different kinds of files, we will start by creating two separate dataframes (one for each set of files). Afterwards, we will homogenize the two dataframes and merge them into a single one.

In [ ]:
"""
Start by importing required stuff:
1. pandas (usually imported as pd)
2. The Path object from the pathlib library
"""
import pandas as pd
from pathlib import Path


"""
Each file will eventually be a row in our consolidated dataframe.
It is *much* faster to build a dataframe from a list of rows, than to append rows to a dataframe.
"""

source1 = []  # Data from all the files whose file name starts with "set"
source2 = []  # Data from all the files consisting of the product ID and the country code

"""
Algorithm: 
1. Go through all the '*.json' files in the directory 'lego-dataset-main/JSON' using the glob() method provided by the Path object
2. Read the file into a temporary Series object (hint: make sure pandas knows to return a Series!)
3. Put the dataframe into the correct container:
    - If the file name starts with "set", it goes into source1
    - Otherwise, the file name needs to be split, the product ID and country code added to the temporary dataframe. Then it goes into source2.
"""

#################### Your code here!


"""
Create two dataframes, one for each source, by concatenating all the temporary Series in the containers.
NB: Make sure we concatenate the Series objects along the columns and we end up with one *row* per Lego set. You may have transpose the concatenated data...
"""

#################### Your code here!

Since the two sets of files came from different sources, we need to make sure that they contain the same kinds of data. The first thing we should check is if there are any columns that are not part of *both* dataframes:

In [ ]:
"""
Algorithm:
1. Get a list of the column names of each DataFrame
2. Calculate the set differences
3. Print out the differences
"""

#################### Your code here!

There are a few columns that are not in both sets. Fortunately, they seem to be just different names for the same kinds of data. We can therefore simply rename the respective columns:

In [ ]:
"""
Use the method 'rename()'.
Hint: This is a good opportunity to use the 'inplace' keyword!
"""

#################### Your code here!

In [ ]:
"""
Check if they are now the same using the same code as above
"""

#################### Your code here!

Now that we have homogenized the structure, we can merge the two dataframes:

In [ ]:
"""
We can use the 'concat()' function again. 

However, pandas will try to retain the indices from the two source dataframes. 
Since we want just a  simple linear index, we can use the method reset_index() to re-enumerate the rows.
"""

#################### Your code here!

Woohoo! All of our Lego sets are in the same data structure!

---

# 2. Analyze Data
With all data now gathered in a single dataframe, we can get some basic statistics on our (uncleaned) dataset:

In [ ]:
"""
Hint: Use the 'describe()' method!
"""

#################### Your code here!

We can see that the columns contain up to 12143 entries. Some of the columns contain numeric values, some contain strings. A few variables are categorical with only a small number of unique levels.

Let's visualize some of the columns to get a general "feel" for the dataset:

In [ ]:
"""
Visualize the distribution of the sets across themes!

Hint: You can count the instances of the unique values in a column using the method 'value_counts()'!
"""

#################### Your code here!

In [ ]:
"""
Create a boxplot of the list prices, grouped by the theme names!
"""

#################### Your code here!

In [ ]:
"""
Create a boxplot of the list prices, grouped by countries!
"""

#################### Your code here!

Now that we have a basic idea of our dataset, we can start looking for inconsistencies. The first thing we should do is check for duplicated rows:

In [ ]:
"""
Use the method 'duplicated()'!

Hint: The method returns a Series of True/False values, one for each row in the dataframe. 
You can additionally use the method 'any()' to quickly check if any row is duplicated!
"""

#################### Your code here!

We have some duplicates in our dataset! We will remove them in the cleaning step.

Next, let's check for missing entries:

In [ ]:
"""
Use the method 'isna()'!
"""

#################### Your code here!

There are indeed missing entries in various columns. Filling missing entries requires case-by-case analysis, which will do during the cleaning step.

There is, of course, a *lot* more to explore (in fact, we offer an entire workshop on Exploratory Data Analysis), but let us move on for now and start cleaning our data!

---

# 3. Clean Data

Let's summarize what kind of dirt we found during the analysis step:

- Some rows are duplicated
- The column `ages` is inconveniently coded
- Some entries are missing

In this step, we will work through this list and clean up the mess.

## 3.1 Duplicated rows

We already now from the analysis that there are some duplicated rows. Let's get a little more specific here and find out how many rows exactly are duplicated:

In [ ]:
"""
Calculate the absolute number of duplicated rows and the relative percentage!
"""

#################### Your code here!

In some contexts, duplicated rows in a dataset can be meaningful (e.g. when sampling an unchanging source repeatedly). In this case, the duplicates are most likely due to our two different data sources, which have some overlap in the scraped Lego sets. We can therefore simply drop the duplicates without losing any information;

In [ ]:
"""
Use the method drop_duplicates()
"""

#################### Your code here!

## 3.2 Convert inconvenient format

Let's take a closer look at the column `'ages'`:

In [ ]:
"""
Print the unique values of the column 'ages'
"""

#################### Your code here!

The encoding of the age range is fairly inconvenient for further analysis because it hides two pieces of information (minimum age and maximum age) in a single string. This makes it unnecessarily difficult to, for example, obtain only the subset of entries that are appropriate for age 13 and above.

We can try to fix this by introducing two new columns: `'age_min'` and `'age_max'`. We can get their respective values by parsing the existing entries. If no maximum value is given (e.g. in `5+`), we will assume `99` (which is a *de-facto* industry standard). First, we homogenize the format of the entries in the column `'ages'`:

In [ ]:
"""
Algorithm:
1. Replace all '+' characters by '-99'
2. Replace '½' by '.5'

Hint: String-related methods are provided using the prefix 'str.'!
"""

#################### Your code here!

Now that all strings have the same format, we can easily split them into the lower and upper boundary and move them into new columns and converting the strings to numbers. Afterwards, we can safely drop the now redundant column `'ages'`.

In [ ]:
"""
Hint: Use the string function 'split()' and the method 'astype("float")'. Make sure the split string is *expanded* into two new columns!
"""

#################### Your code here!

Now we can easily subset our dataset based on arbitrary age ranges:

In [ ]:
"""
Look at the subset of data for minimum age below 9
"""

#################### Your code here!

In [ ]:
"""
Look at the subset of data for maximum age over 12
"""

#################### Your code here!

In [ ]:
"""
Look at the subset of data for the age range 7 to 15
"""

#################### Your code here!

## 3.3 Missing values

As mentioned above, filling in missing values is a case-by-case process. This process can become quite complex and is know as [imputation](https://en.wikipedia.org/wiki/Imputation_(statistics)). Many methods and techniques can be applied to "fill in the gaps" in our data, but before we dive into it ourselves, we need to keep two things in mind:

__Imputed data is not real data!__
No matter how sophisticated our imputation technique is, we need to remind ourselves that the data we are filling in are *not* real-world data! That means, for example, that we should never validate models on datasets containing imputed values. We should also be very carefule when imputing values of a variable that will be used as the target variable in regression or classification model.

__Beware of bias!__
Imputation is data manipulation and as such almost always introduces bias into the dataset. However, not imputing missing values means we would have to drop all data points that are missing values, which may also introduce bias.

In practice, we need to carefully consider our respective scenario, weigh the damage of discarding data versus introducing artificial data, and how intrusive our chosen imputation method may be. 

One important factor to consider is the proportion of missing values. Any imputation needs to be properly supported by a sufficient number of actual data points. We therefore should not try to impute 5000 missing values from only 100 known values. What exactly constitues "enough" support, however, cannot be universally stated.

Let's check our dataset and calculate the proportion of missing values for each column:

In [ ]:
#################### Your code here!

Evidently, we have more than 80 % existing values in each column, which we will consider sufficient support to impute the remaining missing values for this exercise. Therefore, we will not have to drop any columns.

Next, we will go through the columns containing missing values and try to impute them.

### 3.3.1 Number of reviews

The column `num_reviews` contains the number of reviews available for each set. There are quite a few missing values:

In [ ]:
#################### Your code here!

Let's find out which values *are* present in the column:

In [ ]:
"""
Print the minimum, maximum, and average value, as well as the standard deviation using some appropriate methods!
"""

#################### Your code here!

Interesting find: The number of reviews is apparently only recorded when it is greater than 0! We can therefore deduce that the missing values indicate sets that have 0 reviews. 

In this case, we can thus simply replace the missing values with the value 0:

In [ ]:
"""
Hint: Use the method fillna()!
"""

#################### Your code here!

Let's check the statistics again to see how our imputation affected them:

In [ ]:
#################### Your code here!

We can see that the mean value was affected by our imputation, meaning we shifted the distribution of this variable to the left. In this case, that makes perfect sense given that the missing values were not considered before and the mean is now actually more accurate than it was before. In general, however, we have to be very mindful that imputations may introduce distortions of the distribution that are not acceptable!

### 3.3.2 Difficulty

The difficulty of each set is stored in the column `'review_difficulty'`. Let's take a look at the unique levels of this categorical variable and how it is distributed:

In [ ]:
#################### Your code here!

In [ ]:
"""
Hint: Use the method hist() to plot a histogram!
"""

#################### Your code here!

It is reasonable to assume that the piece count has an impact on the difficulty level of a set. So let us also look at the distribution of the piece count broken down by the difficulty level:

In [ ]:
#################### Your code here!

Interesting! We notice fairly distinct distributions for each difficulty level. In other words: Given a specific piece count, the difficulty levels are not equally likely! 
We may also suspect a similar relationship regarding the list price:

In [ ]:
#################### Your code here!

Indeed, the list price shows a similar pattern. 

How about the minimum age?

In [ ]:
#################### Your code here!

Once again, a distinct pattern emerges!


Our brief analysis here suggests that knowing the minimum age, the list price and the number of pieces, we should be able to make a pretty good guess of what the difficulty would be!

To make that guess, we will use Naive Bayes classifier. In a nutshell, this classifier identifies the most likely difficulty level given a minimum age, piece count and list price. While the theory behind this goes beyond the scope of this notebook, let us know if you are interested in this topic and we would be happy to offer a whole workshop on this!

For now, let it suffice to say that our classifier needs to be trained by fitting it to the known pairs of [minimum age, piece count, list price] and corresponding difficulty level. Once it is fitted, the classifier can predict the difficulty level for any combination of minimum age, piece count, and list price.

We will use the package [scikit-learn](https://scikit-learn.org/) to import an implementation of the Naive Bayes classifier:

In [ ]:
"""
Algorithm:

1. Use rows with no missing data as training data for an imputation model
2. Train/fit the model 
3. Predict the missing values for the review difficulty

NB: This is fairly advanced stuff and just here to show you how involved the process of imputing values can get. 
Don't be frustrated if you cannot solve this (yet)!
"""

#################### Your code here!

### 3.3.3 Theme name

There is only a very small number of missing values in the column `'theme_name'`. Let's look at those entries more closely:

In [ ]:
""" 
Print the rows that contain a missing value in the column 'theme_name'

Hint: You can use the output of the method 'isna()' to index the dataframe!
"""

#################### Your code here!

In this case, there are so few entries that we can even consider going back to the data collection stage to find the missing information!

By simply searching for the listed sets on the [Lego website](www.lego.com) ourselves, we can find out the corresponding themes and fill them in manually:

In [ ]:
""" 
Find the missing theme names and fill them in using the '.at[]'. Mind the square brackets!
"""

#################### Your code here!

### 3.3.4 Short descriptions

A few short descriptions are missing from the dataset. Spot checking the affected entries reveals that these sets indeed do not have short descriptions. Therefore, we should replace the missing values with empty strings.

In [ ]:
#################### Your code here!

### 3.3.5 Ratings

The Lego sets are rated by the website users on three different interval scales from 1 (worst) to 5 (best): The *play experience*, the perceived *value*, and the *overall* rating. The columns `play_rating`, `value_rating`, and `overall_rating` contain the respective average rating for each set. Some sets have not been rated on every scale, so some ratings are missing. Let us take a look at the descriptive statistics of the existing ratings. This time, we will use the convenient method `describe()` on the column. Because `describe()` gives us different information depending on the type of the column, we need to convert the columns to a numeric type first:

In [ ]:
#################### Your code here!

We should note that if a set is missing a rating, it is not necessarily missing *all* ratings. 

A reasonable assumption could be that in the absence of a play or value rating, the overall rating is a fairly good stand-in for it. We therefore start by filling in the overall rating for the missing play and/or value rating, if available:

In [ ]:
#################### Your code here!

In [ ]:
#################### Your code here!

As we can see, we were able to fill in a few values without affecting the statistics too much. However, there are still quite a few entries without any ratings at all! 

We should now try to find a best guess for which value to use for each of the missing entries. Our guess could be based on only the univariate statistics of each individual column (e.g. the mean or mode), but that would possibly distort multivariate patterns in the dataset. What if, for example, sets with a large number of pieces tend to have a higher value rating? Ideally, we would try to consider and leverage these relationships between the columns using regression models or *k*-Nearest Neighbor donors. However, this takes us deeper down the rabbit hole of imputation, which is a field in its own right. If you are interested in an advanced workshop on this topic, feel free to contact us so we know that this is of interest!

For the purposes of this notebook, we will simply choose random values for the ratings, drawn from the the existing values. That way we do not distort the existing univariate distribution of each rating, although we possibly weaken existing multivariate patterns in the dataset.

In [ ]:
"""
Algorithm:

1. Get a list of the existing ratings
2. Define a function that accepts the three ratings as an Series and replaces it with random existing ratings, if they are missing.
3. Apply the function to the columns ['play_rating', 'value_rating', 'overall_rating'] of the dataframe using the method 'apply()'.
"""

#################### Your code here!

# 4. Save Data

Our dataset is now fairly tidy! To save us the trouble of going through the entire process again every time we want to do something with this dataset, we can save the cleaned up version to a CSV file.

```python
some_dataFrame.to_csv(filename)
```
</p>


In [ ]:
#################### Your code here!

<p style="font-weight:bold">Congratulations! You have brought order to our messy pile of Legos.</p>

<img src="Images/IMG-lego-messy-to-tidy.png" width="1080px" style="float:center;background-color:white"/>


<a style="float:left;background-color:black;color:white;text-decoration:none;padding:4px 6px;font-family:-apple-system, BlinkMacSystemFont, &quot;San Francisco&quot;, &quot;Helvetica Neue&quot;, Helvetica, Ubuntu, Roboto, Noto, &quot;Segoe UI&quot;, Arial, sans-serif;font-size:12px;font-weight:bold;line-height:1.2;display:inline-block;border-radius:3px" href="https://unsplash.com/@egnaro?utm_medium=referral&amp;utm_campaign=photographer-credit&amp;utm_content=creditBadge" target="_blank" rel="noopener noreferrer" title="Download free do whatever you want high-resolution photos from Rick Mason"><span style="display:inline-block;padding:2px 3px"><span style="display:inline-block;padding:2px 3px">Photo by Rick Mason</span></a><a style="float:right;background-color:black;color:white;text-decoration:none;padding:4px 6px;font-family:-apple-system, BlinkMacSystemFont, &quot;San Francisco&quot;, &quot;Helvetica Neue&quot;, Helvetica, Ubuntu, Roboto, Noto, &quot;Segoe UI&quot;, Arial, sans-serif;font-size:12px;font-weight:bold;line-height:1.2;display:inline-block;border-radius:3px" href="https://www.reddit.com/r/lego/comments/7mhq2b/finished_kind_of_organizing_my_spare_lego_pieces/" target="_blank" rel="noopener noreferrer" title="Download free do whatever you want high-resolution photos from Rick Mason"><span style="display:inline-block;padding:2px 3px"></span><span style="display:inline-block;padding:2px 3px">Photo by rexorhun</span></a>


<div>Icons made by <a href="https://www.flaticon.com/authors/swifticons" title="Swifticons">Swifticons</a> from <a href="https://www.flaticon.com/" title="Flaticon">www.flaticon.com</a></div>

<table >
<tbody>
  <tr>
    <td style="padding:0px;border-width:0px;vertical-align:center">    
    Created by Simon Stone for Dartmouth College Library under <a href="https://creativecommons.org/licenses/by/4.0/">Creative Commons CC BY-NC 4.0 License</a>.<br>For questions, comments, or improvements, email <a href="mailto:researchdatahelp@groups.dartmouth.edu">Research Data Services</a>.
    </td>
    <td style="padding:0 0 0 1em;border-width:0px;vertical-align:center"><img alt="Creative Commons License" src="https://i.creativecommons.org/l/by/4.0/88x31.png"/></td>
  </tr>
</tbody>
</table>
